<a href="https://colab.research.google.com/github/jonbaer/googlecolab/blob/master/YT_Gemini_Context_Caching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U google-generativeai


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.1/163.1 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 717.3/717.3 kB 19.7 MB/s eta 0:00:00


In [ ]:
# Download video file
!wget https://storage.googleapis.com/generativeai-downloads/data/Sherlock_Jr_FullMovie.mp4


--2024-06-20 07:16:17--  https://storage.googleapis.com/generativeai-downloads/data/Sherlock_Jr_FullMovie.mp4
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.157.207, 142.251.8.207, 142.251.170.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.157.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 331623233 (316M) [video/mp4]
Saving to: ‘Sherlock_Jr_FullMovie.mp4.1’

Sherlock_Jr_FullMov 100%[===================>] 316.26M  25.9MB/s    in 13s     

2024-06-20 07:16:30 (24.2 MB/s) - ‘Sherlock_Jr_FullMovie.mp4.1’ saved [331623233/331623233]



In [ ]:
from google.generativeai import caching
import datetime
import time



In [ ]:
from google.colab import output
from IPython.display import display, Markdown
from IPython.display import Image

import google.generativeai as genai

In [ ]:
from google.colab import userdata
GOOGLE_API_KEY= userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
video_file_name = "Sherlock_Jr_FullMovie.mp4"

# Upload the video using the Files API
video_file = genai.upload_file(path=video_file_name)

In [ ]:
# Wait for the file to finish processing
while video_file.state.name == "PROCESSING":
    print('Waiting for video to be processed.')
    time.sleep(2)
    video_file = genai.get_file(video_file.name)

print(f'Video processing complete: {video_file.uri}')

Waiting for video to be processed.
Waiting for video to be processed.
Waiting for video to be processed.
Waiting for video to be processed.
Waiting for video to be processed.
Waiting for video to be processed.
Waiting for video to be processed.
Waiting for video to be processed.
Waiting for video to be processed.
Waiting for video to be processed.
Waiting for video to be processed.
Video processing complete: https://generativelanguage.googleapis.com/v1beta/files/7szod27omkea


In [ ]:
model = genai.GenerativeModel(model_name="models/gemini-1.5-flash-001")

In [ ]:
%%time
normal_call = model.generate_content([video_file,"Tell me about the video and its themes"])

CPU times: user 853 ms, sys: 106 ms, total: 959 ms
Wall time: 1min 13s


In [ ]:
Markdown(normal_call.text)

The video is a clip from the 1924 Buster Keaton film Sherlock Jr. The film is about a young projectionist who dreams of being a detective. In the clip, the projectionist is reading a book about how to be a detective and gets distracted when a woman asks for help finding a lost dollar. He searches the theater for the dollar and uses his detective skills, but ultimately fails to find it. The film explores the themes of fantasy, escapism, and the struggle to achieve one's dreams. 


In [ ]:
print(normal_call.usage_metadata)

prompt_token_count: 696170
candidates_token_count: 105
total_token_count: 696275



In [ ]:
# Create a cache with a 10 minute TTL
cache = caching.CachedContent.create(
    model="models/gemini-1.5-flash-001",
    display_name="sherlock jr movie", # used to identify the cache
    system_instruction="You are an expert video analyzer, and your job is to answer user's query based on the video file you have access to.",
    contents=[video_file],
    ttl=datetime.timedelta(minutes=10),
)

In [ ]:
cache.expire_time

DatetimeWithNanoseconds(2024, 6, 20, 8, 17, 52, 446821, tzinfo=datetime.timezone.utc)

In [ ]:
%%time

# Construct a GenerativeModel which uses the created cache.
model = genai.GenerativeModel.from_cached_content(cached_content=cache)

# Query the model
response = model.generate_content(["Introduce different characters in the movie by describing their personality, looks, and names. Also list the timestamps they were introduced for the first time."])

print(response.usage_metadata)


prompt_token_count: 696218
candidates_token_count: 305
total_token_count: 696523
cached_content_token_count: 696189

CPU times: user 597 ms, sys: 70.5 ms, total: 668 ms
Wall time: 51.2 s


In [ ]:
Markdown(response.text)

Here are some of the main characters in the movie "Sherlock Jr", and how they are presented:

**Buster Keaton:** 
* **First Appearance:** 01:26
* **Personality:** Young, eager, and somewhat naive. He is a film projectionist and aspiring detective. 
* **Looks:** Keaton is known for his deadpan expression and his signature bowler hat. In this film, he wears a white shirt, a dark tie, a dark vest, and pants.

**Kathryn McGuire:**
* **First Appearance:** 02:13
* **Personality:** A young woman who is the daughter of a wealthy family. She is innocent and trusting, and she becomes a target for criminals. 
* **Looks:** Kathryn wears a plaid dress and a black hat. 

**Erwin Connelly:** 
* **First Appearance:** 02:17
* **Personality:**  He is portrayed as a lovable but clumsy character. He is hired to help the girl's father.
* **Looks:**  He wears a dark shirt with suspenders, a white vest, and a hat.

**Ward Crane:** 
* **First Appearance:** 03:41
* **Personality:** A wealthy and arrogant man who is the "sheik" of the town. 
* **Looks:** Ward wears a pinstriped suit, a tie, a shirt, and a hat. He also has a mustache.


In [ ]:
%%time
# Query the model
response = model.generate_content(["What is the last time Buster Keaton appears in the movie?"])

print(response.usage_metadata)

prompt_token_count: 696202
candidates_token_count: 40
total_token_count: 696242
cached_content_token_count: 696189

CPU times: user 513 ms, sys: 52.4 ms, total: 566 ms
Wall time: 42.7 s


In [ ]:
Markdown(response.text)

Buster Keaton appears last at the 4:47 mark. He is seen in a hallway looking at a suit of clothes. 
